# Clone of NanoGPT from Andrej Karpathy

https://github.com/karpathy/ng-video-lecture

## Load data

In [1]:
#download the tiny shakespeare dataset
# !curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > input.txt

In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
chars = sorted(set(text))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Tokeninzer

In [6]:
encoder_map = {c: i for i,c in enumerate(chars) }
decoder_map = {i: c for i,c in enumerate(chars) }

encode = lambda text: [encoder_map[c] for c in text]
decode = lambda encoded_text: ''.join([decoder_map[nb] for nb in encoded_text])

test = "testing :3"
print(encode(test))
print(decode(encode(test)))

[58, 43, 57, 58, 47, 52, 45, 1, 10, 9]
testing :3


### Preprocess dataset 

In [8]:
import torch

encoded_dataset = torch.tensor(encode(text))
print(encoded_dataset.shape, encoded_dataset.dtype)
print(encoded_dataset[:100])

c:\Users\Amine\Documents\Amine\Other\Programmation\AI\LLM\nanoGPT_clone\.venv\lib\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Train test split

In [9]:
train_test_split = (int)(0.9 *len(encoded_dataset))
train_data = encoded_dataset[:train_test_split]
test_data = encoded_dataset[train_test_split:]

Extract target

In [10]:
context_length = 8

In [ ]:
# For each chunk of characters (from 1 to context_length), the target is the next character
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


## Training

Fetch batch

In [41]:
torch.manual_seed(42)

def get_batch(split, batch_size):
  # Generate small of batch of data inputs x and corresponding targets y 
  data = train_data if split=="train" else test_data
  ix = torch.randint(len(data) - context_length, (batch_size,)) #Generate batch_size number (i.e. 4) of random integers between 0 and len(data) - context_length
  x = torch.stack([data[i:i+context_length] for i in ix])
  y = torch.stack([data[i+1:i+context_length+1] for i in ix]) # shift x by 1 to get the corresponding target for each given end index
  return x,y

Example of batch

In [ ]:
xb, yb = get_batch('train', 4)
# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)
# print('----')
# for b in range(batch_size): # batch dimension
#     for t in range(context_length): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

### 1. BigramLanguageModel 

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

# Predict the next character based solely on the previous character
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B=batch_size,T=context_length,C=vocab_size)

        if targets is None:
            loss = None
        else:
            # Reshape logits and targets to be able to apply the pytorch cross entropy loss function
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # Generate max_new_tokens characters by predicting the next character and appending it to the context
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context (i.e. current context of some characters in some batch)
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # extract the logits only for the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [39]:
# Test the model prior to training
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6374, grad_fn=<NllLossBackward0>)

hfGb'zUQdcZLKdvtEz&kbEaSwsZVGvth'UJSCRsbx'UCCD?SHjxma.Xx$oHV!h&ER;,OSLUOnP!VVvXzz&spDICnZIf?zndBnvKw


In [40]:
# Initialize the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

Perform Training

In [46]:
batch_size = 32

for step in range(10000):
  # Sample a batch of data
  xb, yb = get_batch('train', batch_size)
  # Evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(f"loss: {loss.item()}")

loss: 2.440312147140503


In [49]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Car, head mustin'storetldinorakesie, d myove urs kis os nithay.
Pr R:
ATh or maly,


COKETowath ceat
